In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial= 'http://www.investigacionesgeograficas.unam.mx/index.php/rig/issue/archive'
url_root='http://www.investigacionesgeograficas.unam.mx/index.php/rig/issue/archive'
r=requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
box = soup.find('div', class_='issues media-list')
a = box.findAll('div', class_='media-left')
vol = [x.find('a').get('href')for x in a]
vol

In [ ]:
def get_url_items(sopa, url): 
    box = sopa.find('div', class_='issues media-list')
    a = box.findAll('div', class_='media-left')
    vol = [x.find('a').get('href')for x in a]
    return vol

In [ ]:
links_items=[]
i=0
while i<4:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
   list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Investigaciones Geográficas'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    te='Ciencias Sociales, Geografía, Análisis del Territorio'
    ul='http://www.investigacionesgeograficas.unam.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    titulo=s_item.find('h3', class_='media-heading').get_text(strip=True)
    if titulo:
        content_book['Titulo Articulo']=titulo
    else :
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', class_='panel-body').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    urlI=ul
    if urlI:
        content_book['Url Inicial']= urlI
    else :
        content_book['Url Inicial']=None
    #obtener PDF
    ancla_link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
     #lin = [x.get('href')for x in ancla_link]
    if ancla_link:
        content_book['Link Articulo']=ancla_link
    else :
        content_book['Link Articulo']=None
    return content_book

In [ ]:
  list_scraper=list_scraper[0:107]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revist42.csv', index=False)